# Demonstration of Data-handling functions
 - 20191217
 - MJP

In [142]:
# Standard imports 
import os , sys
import numpy as np 
import random
import string
import importlib
import json 


# data code 
sys.path.append(os.path.join(os.path.split(os.getcwd())[0], 'mpchecker2'))
import data
importlib.reload(data)


<module 'data' from '/Users/matthewjohnpayne/Envs/mpcvenv/mpchecker2/mpchecker2/data.py'>

# Background
 - I am expecting that the results of some n-body integration have been encapsulated as a bunch of coefficients
 - The coefficients are all saved in a dictionary 
 - There is a single dictionary for each object
 - I am further assuming that there exists a magic function to use the coefficient-dictionary to evaluate the healpix from the geocenter at a given JD
 - I will demonstrate the data-handling code by creating fake coefficient dictionaries and HP data for a few objects

# --------- Test Code Convenience Functions ---------------------

In [21]:

def randomString(stringLength):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

def sample_DICT():
    ''' 
        Generate a sample dictionary to act as a placeholder for a real coefficient dictionary
    '''
    return { randomString(ri) : np.random.rand(6,8) for ri in np.random.randint(5, 15, size=10) }

def sample_HEALPIX(JDarray , coeff_dict):
    '''
        Generate a sample set of HP to act as a placeholder for a real set of healpix calculated from the coeff-dict
    '''
    return np.random.randint(0, 12*4**5, size=len(JDarray))

def sample_JDintegers(numberJDs):
    '''
        Generate a list/array of integer julian dates to act as a placeholder for a set of JDs of interest
    '''
    JDstart = np.random.randint(2450000, 2458835)
    return np.arange(JDstart, JDstart+numberJDs, 1)

##### Start by creating some sample data ...

In [22]:
name       = 'K19A01B'
coeff_dict = sample_DICT()
JDs        = sample_JDintegers(365)
HP         = sample_HEALPIX(JDs , coeff_dict)

# --------- Data Code Demonstrations ---------------------


### Illustrate some of the variables and methods that are available within the "Data" class

In [50]:
D = data.Data()
D.Names , D._fetch_data_directory()

('names.txt', '/Users/matthewjohnpayne/.mpchecker_data')

### Illustrate some of the variables and methods that are available within the "Names" class

In [132]:
importlib.reload(data)
N = data.Names() 

# Names inherits the Data methods
print(N.Names , N._fetch_data_directory())

# File where name data will be saved ... 
print(N._get_names_filepath())

# For the sake of demo, ensure name-data file does not exist
if os.path.isfile(N._get_names_filepath()):
    os.remove(N._get_names_filepath())
    
# At this point the underlying file is empty / does not exist ...
# ... and so is the names_dict which holds the data
print(N.names_dict)

names.txt /Users/matthewjohnpayne/.mpchecker_data
/Users/matthewjohnpayne/.mpchecker_data/names.txt
OrderedDict([('K19A01B', 0)])


##### Now lets use the sample data created above to start to populate the Names class ...

In [135]:
# insert the name 
i = N.insert(name)
print("The returned value from insert is the position/index of the name: ", i) 
print('The names-dictionary is now populated', N.names_dict, '\n')

# attempt to insert the same name again 
i = N.insert(name)
print("The returned value from insert is the *same* position/index of the name: ", i) 
print("The names-dictionary is unchanged", N.names_dict, '\n')

# write the data to file 
N._write_names_file()

# check that the file now exists and has content
assert os.path.isfile(N._get_names_filepath())
with open(N._get_names_filepath(), 'r') as fp:
    print(fp.readlines())

The returned value from insert is the position/index of the name:  0
The names-dictionary is now populated OrderedDict([('K19A01B', 0)]) 

The returned value from insert is the *same* position/index of the name:  0
The names-dictionary is unchanged OrderedDict([('K19A01B', 0)]) 

['K19A01B\n']


##### Now that the above example has populated the names-file on disk, we can read from it

In [136]:
N = data.Names() 
print('After re-initialize: names-dict populated as file is read by default: ', N.names_dict, '\n')


After re-initialize: names-dict populated as file is read by default:  OrderedDict([('K19A01B', 0)]) 



### Illustrate some of the variables and methods that are available within the "ObjectCoefficients" class

In [182]:
importlib.reload(data)

# Initialize an ObjectCoefficients object  
OC = data.ObjectCoefficients()

# Look at the 'coeff_list' variable: it should be an empty list at this point
print('Empty coeff-list:\n\t', OC.coeff_list)

# Note the OC has access to the names_dict from the Names object
print('Names dict is available:\n\t', OC.names_dict)

# Now insert an arbitrary dictionary as if it was a reasonable coefficient set ...
OC.upsert(name, coeff_dict)
print('After upcert, co-effs populated:\n\t',OC.coeff_list)
 
# For the sake of demo, ensure coeff-data file does not exist
if os.path.isfile(OC._get_coeff_filepath()):
    os.remove(OC._get_coeff_filepath())

# write the data to file **NOT WRITING PROPERLY ***
OC._write_coeff_file()

# check that the file now exists and has content
assert os.path.isfile(OC._get_coeff_filepath())
print('file has content ...\n\t', OC._read_coeff_file() )


Empty coeff-list:
	 []
Names dict is available:
	 OrderedDict([('K19A01B', 0)])
After upcert, co-effs populated:
	 [{'avkxhuhiaqe': array([[0.57575568, 0.40806314, 0.68768492, 0.98457724, 0.87720618,
        0.08126853, 0.94974897, 0.22902447],
       [0.62559518, 0.43610831, 0.96866208, 0.857295  , 0.44955387,
        0.55459604, 0.36496351, 0.17834266],
       [0.32131966, 0.90214858, 0.56584599, 0.4488772 , 0.23464336,
        0.36948072, 0.57705926, 0.88530183],
       [0.71632793, 0.18336061, 0.25987418, 0.98931489, 0.71100215,
        0.43411904, 0.20756345, 0.22985702],
       [0.16793146, 0.76417497, 0.82345838, 0.41822396, 0.20387469,
        0.64286072, 0.91306912, 0.36208742],
       [0.73095507, 0.25124433, 0.25353087, 0.62078124, 0.77324681,
        0.99319553, 0.89794186, 0.21948617]]), 'lpoedzljywnl': array([[0.58729799, 0.61880727, 0.26305895, 0.01927637, 0.78621227,
        0.05005631, 0.57219354, 0.07017154],
       [0.86817138, 0.90990294, 0.58056006, 0.61312412, 0.2

##### Now that the coefficient file has been populated, it will be read directly when a coefficient object is declared:

In [184]:
importlib.reload(data)

# Initialize an ObjectCoefficients object  
OC = data.ObjectCoefficients()

# Look at the 'coeff_list' variable: it should be an populated now (because of the above cell)
print('Populated coeff-list:\n\t', OC.coeff_list)


Populated coeff-list:
	 [{'avkxhuhiaqe': [[0.5757556810259852, 0.4080631417658168, 0.6876849152344771, 0.9845772387850682, 0.8772061822356455, 0.08126852900040071, 0.9497489672904957, 0.2290244688641091], [0.6255951798971503, 0.43610830989876836, 0.9686620786036545, 0.8572949978444568, 0.44955386992138546, 0.5545960426363966, 0.36496351055886245, 0.17834265568939278], [0.32131965556282027, 0.9021485784412708, 0.565845986786536, 0.44887720312670054, 0.23464336151846477, 0.36948072153856304, 0.5770592580973724, 0.8853018317187517], [0.7163279301707844, 0.18336061098942102, 0.25987417527100154, 0.9893148943022023, 0.7110021500903051, 0.43411904384974187, 0.20756344686340977, 0.22985702004792075], [0.16793146212280163, 0.7641749734409313, 0.8234583762807123, 0.41822396441876686, 0.20387469209242448, 0.6428607212219328, 0.9130691238662614, 0.3620874166359531], [0.7309550673044083, 0.25124433345364805, 0.2535308747700271, 0.6207812358760646, 0.7732468108586633, 0.9931955289807023, 0.89794185

##### Demonstration of code to read & write HEALPIX coefficients 